In [2]:
import tensorflow as tf
from tensorflow import keras
import numpy as np



c:\Users\jorge.ruiz\Anaconda3\envs\quant\lib\site-packages\flatbuffers\compat.py:19: DeprecationWarning: the imp module is deprecated in favour of importlib and slated for removal in Python 3.12; see the module's documentation for alternative uses
  import imp
c:\Users\jorge.ruiz\Anaconda3\envs\quant\lib\site-packages\keras\utils\image_utils.py:36: DeprecationWarning: NEAREST is deprecated and will be removed in Pillow 10 (2023-07-01). Use Resampling.NEAREST or Dither.NONE instead.
  'nearest': pil_image.NEAREST,
c:\Users\jorge.ruiz\Anaconda3\envs\quant\lib\site-packages\keras\utils\image_utils.py:37: DeprecationWarning: BILINEAR is deprecated and will be removed in Pillow 10 (2023-07-01). Use Resampling.BILINEAR instead.
  'bilinear': pil_image.BILINEAR,
c:\Users\jorge.ruiz\Anaconda3\envs\quant\lib\site-packages\keras\utils\image_utils.py:38: DeprecationWarning: BICUBIC is deprecated and will be removed in Pillow 10 (2023-07-01). Use Resampling.BICUBIC instead.
  'bicubic': pil_image.

In [3]:
from typing import TypedDict


class DataType(TypedDict):
    X_train: np.ndarray
    X_test: np.ndarray
    Y_train: np.ndarray
    Y_test: np.ndarray

In [4]:
data: DataType = np.load("data.npy")
for label, data_ in data.items():
    print(f"{label}: {data_.shape}")



X_train: (4404, 9)
X_test: (2170, 9)
Y_train: (4404,)
Y_test: (2170,)


In [5]:


model = keras.Sequential([
    tf.keras.Input(shape=data["X_train"].shape[1:], name="input"),
    keras.layers.Dense(10, name="dense1", activation="tanh"),
    keras.layers.Dense(2**4, name="dense2", activation="tanh"),
    keras.layers.Dense(2**5, name="dense3", activation="tanh"),
    keras.layers.Dense(2**2, name="dense4", activation="tanh"),
    keras.layers.Dense(1, name="output")

])

model.compile(optimizer="adam", loss = 'mse')
model.summary()


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense1 (Dense)              (None, 10)                100       
                                                                 
 dense2 (Dense)              (None, 16)                176       
                                                                 
 dense3 (Dense)              (None, 32)                544       
                                                                 
 dense4 (Dense)              (None, 4)                 132       
                                                                 
 output (Dense)              (None, 1)                 5         
                                                                 
Total params: 957
Trainable params: 957
Non-trainable params: 0
_________________________________________________________________


In [6]:
#print(np.expand_dims(data['X_test'], 0).shape)
history = model.fit(x=data['X_train'], y = data['Y_train'], epochs=100, 
    validation_data=(data['X_test'], data['Y_test']),
     batch_size=32)

Epoch 1/100
138/138 [==============================] - 1s 3ms/step - loss: 0.0414 - val_loss: 0.0257
Epoch 2/100
138/138 [==============================] - 0s 2ms/step - loss: 0.0247 - val_loss: 0.0253
Epoch 3/100
138/138 [==============================] - 0s 3ms/step - loss: 0.0243 - val_loss: 0.0260
Epoch 4/100
138/138 [==============================] - 0s 2ms/step - loss: 0.0242 - val_loss: 0.0251
Epoch 5/100
138/138 [==============================] - 0s 3ms/step - loss: 0.0241 - val_loss: 0.0246
Epoch 6/100
138/138 [==============================] - 0s 3ms/step - loss: 0.0242 - val_loss: 0.0254
Epoch 7/100
138/138 [==============================] - 0s 2ms/step - loss: 0.0242 - val_loss: 0.0248
Epoch 8/100
138/138 [==============================] - 0s 3ms/step - loss: 0.0240 - val_loss: 0.0247
Epoch 9/100
138/138 [==============================] - 0s 2ms/step - loss: 0.0238 - val_loss: 0.0248
Epoch 10/100
138/138 [==============================] - 0s 2ms/step - loss: 0.0237 - val_lo

In [7]:
model.evaluate(data['X_test'], data['Y_test'])

68/68 [==============================] - 0s 1ms/step - loss: 0.0222


0.022209661081433296

In [9]:
import pickle
import sklearn 
f = open("./wind_scaler.pkl", "rb")
wind_scaler = pickle.load(f)
f.close()

predictions = wind_scaler.inverse_transform(model.predict(data['X_test']))
expected = wind_scaler.inverse_transform(data['Y_test'].reshape(-1, 1))

for pred, real in zip(predictions, expected):
    print(pred, real)
    

68/68 [==============================] - 0s 1ms/step


C:\Users\jorge.ruiz\AppData\Local\Temp\ipykernel_1680\226785554.py:3: ResourceWarning: unclosed file <_io.BufferedReader name='./wind_scaler.pkl'>
  f = open("./wind_scaler.pkl", "rb")
c:\Users\jorge.ruiz\Anaconda3\envs\quant\lib\site-packages\sklearn\base.py:329: UserWarning: Trying to unpickle estimator MinMaxScaler from version 1.0.2 when using version 1.1.1. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(


[12.210108] [20.12]
[9.318446] [14.92]
[9.264759] [14.79]
[12.271356] [8.46]
[10.379878] [5.75]
[8.372251] [11.08]
[6.5393105] [8.]
[11.648505] [15.21]
[10.750166] [6.]
[10.686962] [12.87]
[10.392014] [10.63]
[9.5146885] [8.58]
[13.458073] [20.21]
[10.375932] [8.96]
[7.9022875] [2.58]
[8.486463] [11.96]
[5.9642577] [3.08]
[11.943932] [11.34]
[7.20008] [3.]
[9.027395] [10.58]
[7.0910087] [3.42]
[6.153326] [8.58]
[8.412953] [13.79]
[9.882583] [6.25]
[8.016321] [7.04]
[9.759398] [6.29]
[13.165877] [10.92]
[9.21304] [9.17]
[11.667462] [1.08]
[10.991946] [8.79]
[8.027665] [13.29]
[8.526767] [12.04]
[9.413644] [12.21]
[6.640333] [5.5]
[11.238914] [4.38]
[8.270157] [6.46]
[8.963172] [6.87]
[13.816647] [10.29]
[10.351401] [8.25]
[8.359927] [4.46]
[9.702279] [11.63]
[8.304792] [12.12]
[16.920673] [22.]
[7.7547197] [11.54]
[8.97693] [2.21]
[10.157977] [7.67]
[8.147276] [6.83]
[10.112211] [10.]
[8.693936] [1.63]
[7.9680886] [13.17]
[11.170027] [12.42]
[12.051201] [16.54]
[10.77501] [8.38]
[9.2372

In [10]:
model.save("model.h5")